In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import category_encoders as ce
import statsmodels.formula.api as sm
import seaborn as sns
%matplotlib inline

pd.set_option('display.max_rows', 300)

In [2]:
df_train = pd.read_csv('/kaggle/input/house-price-dataset/train (1).csv')

In [3]:
df_train =  pd.get_dummies(df_train,drop_first=True)
best_corrs = df_train.corr()['SalePrice'].sort_values(ascending=False).drop(['SalePrice'],axis=0).reset_index(drop=True).head(2)
best_corrs

0    0.790982
1    0.708624
Name: SalePrice, dtype: float64

In [4]:
#Encoding every value that it`s type is equal to object, so we can use it in our regression
cols = []
for i in df_train.columns:
    if(df_train[i].dtype == 'object'):
        cols.append(i)
encoder = ce.OrdinalEncoder(cols=cols)
X_train =  encoder.fit_transform(df_train.drop(['SalePrice'],axis=1).reset_index(drop=True))
Y_train = df_train['SalePrice']

In [5]:
model = sm.ols(formula='SalePrice ~ OverallQual + GrLivArea ',data=encoder.fit_transform(df_train.drop(['Id'],axis=1).reset_index(drop=True)))
model = model.fit()
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              SalePrice   R-squared:                       0.714
Model:                            OLS   Adj. R-squared:                  0.714
Method:                 Least Squares   F-statistic:                     1820.
Date:                Tue, 05 Nov 2024   Prob (F-statistic):               0.00
Time:                        23:41:25   Log-Likelihood:                -17630.
No. Observations:                1460   AIC:                         3.527e+04
Df Residuals:                    1457   BIC:                         3.528e+04
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
===============================================================================
                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------
Intercept   -1.041e+05   5045.372    -20.631      0.000   -1.14e+05   -9.42e+04
OverallQual  3.285e+04    999.198     32.875      0.000    3.09e+04    3.48e+04
GrLivArea      55.8622      2.630     21.242      0.000      50.704      61.021
==============================================================================
Omnibus:                      341.985   Durbin-Watson:                   1.985
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             8725.150
Skew:                           0.469   Prob(JB):                         0.00
Kurtosis:                      14.939   Cond. No.                     7.35e+03
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 7.35e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [6]:
df_test = pd.read_csv('/kaggle/input/house-price-dataset/test (1).csv') 


In [7]:
X_test = encoder.fit_transform(df_test)


In [8]:
predictions = model.predict(X_test)

In [9]:
submission = pd.DataFrame({'Id':df_test['Id'],'SalePrice':predictions})

In [10]:
submission.head()
submission.to_csv('./submission.csv',index=False)  